In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.argv=['']
del sys
os.chdir("../")


In [53]:
import os
import json
import math
import torch
import numpy as np
import pandas
import logging


from IPython import embed
from common import data_preprocess 
from common.utils import print_to_json
from common.dataloader import load_SMAP_MSL_dataset, load_CSV_dataset
from common.sliding import BatchSlidingWindow, WindowIterator
from common.config import parse_arguments, set_logger, initialize_config



In [12]:
args = parse_arguments()

# load config
config_dir = "./hypers/" if not args["load"] else args["load"]
params = initialize_config(config_dir, args)

# load & preprocess data
data_dict = load_SMAP_MSL_dataset(params["path"], params["dataset"])
pp = data_preprocess.preprocessor()
if params["discretized"]:
    data_dict = pp.discretize(data_dict)
    vocab_size = pp.build_vocab(data_dict)
pp.save(params["save_path"])

train_windows, test_windows, test_labels = data_preprocess.generate_windows(data_dict, window_size=params["window_size"])

train_iterator = WindowIterator(train_windows, batch_size=params["batch_size"], shuffle=True)
test_iterator = WindowIterator(test_windows, batch_size=params["batch_size"], shuffle=False)
params['in_channels'] = data_dict["dim"]

2020-11-19 14:53:49,441 P73779 INFO Saving preprocessor into ./checkpoints/20201119-145349/preprocessor.pkl
2020-11-19 14:53:49,443 P73779 INFO Generating sliding windows for dataset [SMAP]


In [60]:
test_labels = test_labels[:, -1]

In [61]:
sum(test_labels)

tensor(54696.)

In [62]:
sum(test_labels).item() / test_labels.shape[0]

0.12792201565580613

In [14]:
inp = next(iter(test_iterator.loader))

In [51]:
inp.numpy()[0:None].shape

(64, 25, 45)

In [46]:
from networks.mlstm import MultiLSTMEncoder

In [47]:
model = MultiLSTMEncoder(**params)

In [42]:
oup=model.encoder(inp)

AttributeError: 'MultiLSTMEncoder' object has no attribute 'encoder'

In [ ]:
logging.info("Proceeding using {}...".format(params["device"]))
logging.info(print_to_json(params))
# training
encoder = MultiLSTMEncoder(vocab_size=vocab_size, **params)

if params["load"]:
    encoder.load_encoder()
else:
    encoder.fit(train_iterator, save_memory=False)
    encoder.save_encoder()

# inference
features = encoder.encode(test_iterator.loader)
logging.info("Final features have shape: {}".format(features.shape))



In [56]:
a = np.array([0,0.1,0.12,0.15,0.9])
np.percentile(a, 90)

0.6000000000000001

In [79]:
import h5py

def load_hdf5(infile,):
    with h5py.File(infile, 'r') as f:
        return {key : f[key][:] for key in list(f.keys())}

def write_hdf5(outfile, arr_dict):
    with h5py.File(outfile, 'w') as f:
        for key in arr_dict.keys():
            f.create_dataset(key, data=arr_dict[key])

In [86]:
a = torch.randn(5,6)

In [87]:
write_hdf5("./save_test1", {"a": a})

In [88]:
load_hdf5("./save_test1")

{'a': array([[-0.42135376,  0.5397852 ,  0.4116953 ,  0.7940437 , -1.5295442 ,
         -0.28173816],
        [ 1.0826707 ,  0.05875599,  0.48668915, -0.7156641 , -1.2892246 ,
         -1.4234002 ],
        [ 0.48751888,  1.2072687 , -1.6384157 , -1.320613  , -1.1679837 ,
          0.4241657 ],
        [-0.80352515, -0.13083345, -0.03963302,  0.4198667 , -0.14734846,
          0.76326144],
        [-1.5251911 ,  1.4436054 ,  0.24256584,  0.29691985, -0.95309186,
          0.02422922]], dtype=float32)}